<a href="https://colab.research.google.com/github/pgupta1980/ai-agents-for-beginners/blob/main/02-explore-agentic-frameworks/code_samples/02-autogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoGen Basic Sample

In this code sample, you will use the [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework to create a basic agent.

The goal of this sample is to show you the steps that we will later use in the addtional code samples when implementing the different agentic patterns.

## Import the Needed Python Packages

In [4]:
!pip install autogen pip autogen-agentchat autogen-ext[openai] autogen-ext[azure]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.2/188.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 5.6 MB/s eta 0:00:00


In [5]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Create the Client

In this sample, we will use [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) for access to the LLM.

The `model` is defined as `gpt-4o-mini`. Try changing the model to another model available on the GitHub Models marketplace to see the different results.

As a quick test, we will just run a simple prompt - `What is the capital of France`.

In [6]:
from google.colab import userdata
import os
os.environ["GITHUB_TOKEN"] = userdata.get('GITHUB_TOKEN')

In [7]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=14, completion_tokens=7) cached=False logprobs=None thought=None


## Defining the Agent

Now that we have set up the `client` and confirmed that it is working, let us create an `AssistantAgent`. Each agent can be assigned a:
**name** - A short hand name that will be useful in referencing it in multi-agent flows.
**model_client** - The client that you created in the earlier step.
**tools** - Available tools that the Agent can use to complete a task.
**system_message** - The metaprompt that defines the task, behavior and tone of the LLM.

You can change the system message to see how the LLM responds. We will cover `tools` in Lesson #4.


In [11]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Run the Agent

The below function will run the agent. We use the the `on_message` method to update the Agent's state with the new message.

In this case, we update the state with a new message from the user which is `"Plan me a great sunny vacation"`.

You can change the message content to see how the LLM responds differently.

In [9]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="Plan me a great sunny vacation", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

[]
source='assistant' models_usage=RequestUsage(prompt_tokens=26, completion_tokens=675) metadata={} content="Sure! Here’s a plan for a sunny vacation that offers relaxation, adventure, and beautiful scenery. Let's consider a 7-day getaway to San Diego, California, which is known for its stunning beaches, vibrant culture, and pleasant weather year-round.\n\n### Destination: San Diego, California\n\n#### Day 1: Arrival\n- **Morning:** Arrive at San Diego International Airport and check into your hotel (recommendations: Hotel del Coronado for a luxury experience or a beachfront Airbnb).\n- **Afternoon:** Relax at Coronado Beach. Enjoy the sun and beautiful views of the Pacific Ocean.\n- **Evening:** Dinner at Peohe's, a waterfront restaurant with stunning sunset views.\n\n#### Day 2: Explore Balboa Park\n- **Morning:** Breakfast at The Cottage in La Jolla. \n- **Day:** Spend the day at Balboa Park. Visit the San Diego Zoo (highly recommended) and explore various museums such as the Museu

In [13]:
from pprint import pprint

async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="Plan me a great windy and haunted vacation", source="user")],
        cancellation_token=CancellationToken(),
    )
    # print(response.inner_messages)
    # print(response.chat_message)
    pprint(response.inner_messages)
    pprint(response.chat_message)

# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

# pprint(agent.state_manager.state)


[]
TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=704, completion_tokens=667), metadata={}, content="Absolutely! Here’s a thrilling itinerary for a vacation that features both windy coastal landscapes and haunted locales. \n\n### Destination: **Savannah, Georgia, and Tybee Island**\n\n#### Duration: **5 Days**\n\n---\n\n### Day 1: Arrival in Savannah\n- **Flight:** Arrive at Savannah/Hilton Head International Airport.\n- **Accommodation:** Check into a historic (and potentially haunted) hotel like the **Belford Inn** or **The Marshall House**.\n- **Evening:** Take a leisurely stroll along the Savannah Riverwalk. Dinner at **The Olde Pink House**, a historic restaurant with a ghostly reputation.\n\n### Day 2: Savannah’s Historic District\n- **Morning:** Begin your day with a guided tour of the Savannah Historic District. Visit Waving Girl Statue and Forsyth Park.\n- **Afternoon:** Join a ghost tour like **Savannah Ghost Tours** or **Haunted History Tours**; expl